# 2-Table Assembling

This notebook assembles the results of the [scraping](https://github.com/njparker1993/oscars_predictions/blob/master/scraping.ipynb) into one dataframe into a machine learning-ready DataFrame.

In [1]:
import pandas as pd
import numpy as np

In [2]:
osc_scrape = pd.read_csv('./data/scraping_results/osc_bp.csv')
noms = pd.read_csv('./data/scraping_results/noms.csv')
gg_drama = pd.read_csv('./data/scraping_results/gg_drama.csv')
gg_comedy = pd.read_csv('./data/scraping_results/gg_comedy.csv')
pga = pd.read_csv('./data/scraping_results/pga.csv')
bafta = pd.read_csv('./data/scraping_results/bafta.csv')
dga = pd.read_csv('./data/scraping_results/dgas.csv')
sag = pd.read_csv('./data/scraping_results/sag_ensemble.csv')
cannes = pd.read_csv('./data/scraping_results/cannes.csv')

# Join on Nominations

In [3]:
# The Nominations Table doesn't cover 2019 so I will add these films manually
osc_2019 = osc_scrape.loc[(osc_scrape['year'] == 2019)]
osc_2019.film

554                   Ford v Ferrari
555                     The Irishman
556                      Jojo Rabbit
557                Joker (2019 film)
558         Little Women (2019 film)
559                   Marriage Story
560                 1917 (2019 film)
561    Once Upon a Time in Hollywood
562             Parasite (2019 film)
Name: film, dtype: object

In [4]:
# I use this order and manually fill in the nominations for this year
nom_count_2019 = [4,10,6,11,6,6,10,10,6]
osc_2019['Nominations'] = nom_count_2019
noms = pd.concat([noms,osc_2019])
noms_merge = noms[['film','Nominations']].drop_duplicates()
osc_scrape = pd.merge(osc_scrape, noms_merge, on = 'film', how = 'left')
osc_to_avg = osc_scrape.copy()
avg_noms = int(osc_to_avg.loc[(~osc_to_avg['Nominations'].isna())]['Nominations'].mean())
osc_scrape.loc[(osc_scrape['Nominations'].isna()), 'Nominations'] = avg_noms

/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


## Join on the other awards shows

In [5]:
def table_assemble(main_df, to_add_df, show_name):
    """
    Given an awards show scraped from Wikipedia, this function
    Adds it as a feature as a 0/1 flag if it was
    Nominated for that Award and if it won
    """
    nom_col = str('nom_') + show_name
    win_col = str('winner_') + show_name
    # Initalize Columns as no
    main_df[nom_col] = 0
    main_df[win_col] = 0
    to_add_df.columns = ['year','film','wiki','winner_add']
    main_df = pd.merge(main_df, to_add_df[['film','winner_add']], on = 'film', how = 'left')
    main_df.loc[(~main_df['winner_add'].isna()), nom_col] = 1
    main_df.loc[(main_df['winner_add'] == True), win_col] = 1
    main_df = main_df.drop('winner_add', axis = 1)
    return main_df


In [6]:
# Adding a response columns
osc_scrape['Oscar_win'] = 0
osc_scrape.loc[(osc_scrape['winner'] == True), 'Oscar_win'] = 1

In [7]:
# Adding on the results of each festival
scraped_dfs = [gg_drama, gg_comedy, pga, bafta, dga, sag, cannes]
scraped_names = ['gg_drama','gg_comedy','pga', 'bafta', 'dga', 'sag', 'cannes']

for i in range(len(scraped_dfs)):
    osc_scrape = table_assemble(osc_scrape, scraped_dfs[i], scraped_names[i])

,year,film,wiki,winner,Nominations,Oscar_win,nom_gg_drama,winner_gg_drama,nom_gg_comedy,winner_gg_comedy,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
0,1927,Wings (1927 film),/wiki/Wings_(1927_film),True,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1927,The Racket (1928 film),/wiki/The_Racket_(1928_film),False,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1927,7th Heaven (1927 film),/wiki/7th_Heaven_(1927_film),False,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1928,Alibi (1929 film),/wiki/Alibi_(1929_film),False,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,2019,Little Women (2019 film),/wiki/Little_Women_(2019_film),False,6.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
559,2019,Marriage Story,/wiki/Marriage_Story,False,6.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
560,2019,1917 (2019 film),/wiki/1917_(2019_film),False,10.0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0
561,2019,Once Upon a Time in Hollywood,/wiki/Once_Upon_a_Time_in_Hollywood,False,10.0,0,0,0,1,1,1,0,1,0,1,0,1,0,1,0


In [8]:
# Some awards didn't exist until certain years
for i in range(len(scraped_dfs)):
    print(scraped_names[i], scraped_dfs[i].year.min())

gg_drama 1943
gg_comedy 1951
pga 1989
bafta 1960
dga 1948
sag 1995
cannes 1970


In [9]:
# Saving the data
osc_scrape.to_csv('./data/processed_results/osc_df')

# Results
The DataFrame is now Machine Learning ready, with 0's and 1's as markers for how a given Oscar-Nominated Film did for that Awards show. The response column will be Oscar_win. An example of this final DataFrame can be seen below.

In [18]:
cols = list(osc_scrape.columns)
display_cols = cols[0:2] + cols[4:]
osc_scrape[display_cols].sample(5)

,year,film,Nominations,Oscar_win,nom_gg_drama,winner_gg_drama,nom_gg_comedy,winner_gg_comedy,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
213,1958,The Defiant Ones,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
374,1990,Goodfellas,6.0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0
66,1936,San Francisco (1936 film),6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
136,1943,The More the Merrier,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
457,2007,Juno (film),4.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
